In [1]:
# Imports
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('./data/processed_data.csv')
df.shape


(18471, 69)

In [3]:
# Final labels
labels = ['patientState_good', 'patientState_caution', 'patientState_danger']
# Section labels
section_labels = {
    'fever': ['feverState_good', 'feverState_caution', 'feverState_danger'],
    'medication': ['medicationState_good', 'medicationState_caution', 'medicationState_danger'],
    'hydration': ['hydrationState_good', 'hydrationState_caution', 'hydrationState_danger'],
    'respiration': ['respirationState_good', 'respirationState_caution', 'respirationState_danger'],
    'skin': ['skinState_good', 'skinState_caution', 'skinState_danger'],
    'pulse': ['pulseState_good', 'pulseState_caution', 'pulseState_danger'],
    'general': ['generalState_good', 'generalState_caution', 'generalState_danger'],
}
# Section labels in an array flattened
section_labels_arr = [item for val in section_labels.values() for item in val]
# All labels
all_labels = [*labels, *section_labels_arr]
len(all_labels)


24

In [4]:
df[section_labels['general']].value_counts()


generalState_good  generalState_caution  generalState_danger
1.0                0.0                   0.0                    10142
0.0                1.0                   0.0                     7976
                   0.0                   1.0                      353
dtype: int64

In [5]:
# Separating to testing and training

# Prepare the data: You should split your data into training and test sets.
# The training set will be used to train the model and the test set will be used
# to evaluate the model's performance.

_x, x_test, _y, y_test = train_test_split(
    df.drop(columns=all_labels).to_numpy(),
    df[labels].to_numpy(),
    test_size=0.2,
    random_state=42
)
x_train, x_val, y_train, y_val = train_test_split(
    _x,
    _y,
    test_size=0.25,
    train_size=0.75,
    random_state=42
)
x_train.shape


(11082, 45)

In [6]:
str(list(x_test[1]))


'[15.6173706445342, 1.0, 38.3, 0.0, 0.0, 0.0, 1.0, 1.0, 50.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 13.0, 0.0, 0.0, 0.0, 0.0, 68.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]'

In [7]:
# Preprocess the data: Data preprocessing is an important step, which includes
# cleaning and transforming the data. You should normalize the data,
# one-hot encode categorical variables, and split the data into features and labels.

# TODO: Does KNNImputer create a correlation between training and test split?
# If yes do ->
# TODO: move KNNImputer here
# TODO: move every preprocessing step that would create a connection between train and test split


In [16]:
from tensorflow import keras


In [17]:
# Create a baseline model to measure performance
b_model = keras.Sequential([
    keras.layers.Dense(units=45, activation='tanh', input_shape=(45,)),
    keras.layers.Dense(units=3, activation='softmax'),
])
b_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
b_model.build()
b_model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 45)                2070      
                                                                 
 dense_4 (Dense)             (None, 3)                 138       
                                                                 
Total params: 2,208
Trainable params: 2,208
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Train baseline model
NUM_EPOCHS = 20

# Early stopping set after 5 epochs
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

b_model.fit(
    x_train,
    y_train,
    epochs=NUM_EPOCHS,
    validation_data=(x_val, y_val),
    callbacks=[stop_early],
    verbose=2
)

Epoch 1/20


2023-02-24 11:08:18.267617: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-24 11:08:24.822667: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


347/347 - 9s - loss: 0.7624 - accuracy: 0.6552 - val_loss: 0.6669 - val_accuracy: 0.7290 - 9s/epoch - 25ms/step
Epoch 2/20
347/347 - 7s - loss: 0.6013 - accuracy: 0.7672 - val_loss: 0.5586 - val_accuracy: 0.7856 - 7s/epoch - 20ms/step
Epoch 3/20
347/347 - 7s - loss: 0.5154 - accuracy: 0.8069 - val_loss: 0.5142 - val_accuracy: 0.7888 - 7s/epoch - 20ms/step
Epoch 4/20
347/347 - 8s - loss: 0.4640 - accuracy: 0.8284 - val_loss: 0.4540 - val_accuracy: 0.8270 - 8s/epoch - 22ms/step
Epoch 5/20
347/347 - 7s - loss: 0.4334 - accuracy: 0.8396 - val_loss: 0.4365 - val_accuracy: 0.8392 - 7s/epoch - 21ms/step
Epoch 6/20
347/347 - 7s - loss: 0.4028 - accuracy: 0.8498 - val_loss: 0.3983 - val_accuracy: 0.8508 - 7s/epoch - 20ms/step
Epoch 7/20
347/347 - 7s - loss: 0.3808 - accuracy: 0.8589 - val_loss: 0.3909 - val_accuracy: 0.8506 - 7s/epoch - 20ms/step
Epoch 8/20
347/347 - 7s - loss: 0.3659 - accuracy: 0.8653 - val_loss: 0.3905 - val_accuracy: 0.8541 - 7s/epoch - 20ms/step
Epoch 9/20
347/347 - 8s - l

In [30]:
def evaluate_model(model, x_test, y_test):
    """
    evaluate model on test set and show results in dataframe.
    
    Parameters
    ----------
    model : keras model
        trained keras model.
    X_test : numpy array
        Features of holdout set.
    y_test : numpy array
        Labels of holdout set.
        
    Returns
    -------
    display_df : DataFrame
        Pandas dataframe containing evaluation results.
    """
    eval_dict = model.evaluate(x_test, y_test, return_dict=True)
    
    display_df = pd.DataFrame([eval_dict.values()], columns=[list(eval_dict.keys())])
    
    return display_df

# Evaluate model on test set and add results to dataframe
results = evaluate_model(b_model, x_test, y_test)

# Set index to 'Baseline'
results.index = ['Baseline']

# Display results
results.head()

116/116 [==============================] - 2s 16ms/step - loss: 0.2859 - accuracy: 0.8955


,loss,accuracy
Baseline,0.285855,0.895535


In [8]:
# Define the model: TensorFlow provides a high-level API for building and
# training neural network models. You should choose the appropriate model
# architecture for your problem and specify the hyperparameters,
# such as the number of hidden layers and the number of neurons in each layer.

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='./checkpoints/model_{epoch}',
        save_freq='epoch'),
    keras.callbacks.TensorBoard(log_dir='./logs')
]


def build_model(hp):
    """
    Builds model and sets up hyperparameter space to search.
    
    Parameters
    ----------
    hp : HyperParameter object
        Configures hyperparameters to tune.
        
    Returns
    -------
    model : keras model
        Compiled model with hyperparameters to tune.
    """
    model = keras.Sequential([
        keras.layers.Dense(
            units=hp.Int('units', min_value=32, max_value=512, step=32),
            activation=hp.Choice('activation', values=['tanh', 'sigmoid', 'relu']),
            input_shape=(45,)),
    ])
    layey_two = hp.Boolean('2nd_layer')
    with hp.conditional_scope('2nd_layer', True):
        if (layey_two):
            model.add(
                keras.layers.Dense(
                    units=hp.Int('units2', min_value=32, max_value=512, step=32),
                    activation=hp.Choice('activation', values=['tanh', 'sigmoid', 'relu']))
            )
    # ? should I include a dropout layer?
    # ? should I normalize the data? If so what happens with new predictions
    model.add(
        keras.layers.Dense(3, activation='softmax')
    )
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        # optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    model.build()
    return model


In [11]:
# Compile the model: You should compile the model by specifying the optimizer,
# loss function, and evaluation metrics.
import keras_tuner

tuner = keras_tuner.tuners.Hyperband(
    build_model,
    objective='val_loss',
    # max_trials=50,
    executions_per_trial=2,
    overwrite=False,
    directory='tuner2')


In [12]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(x_train, y_train, validation_data=(x_val, y_val), epochs=4, callbacks=[stop_early])


Trial 254 Complete [00h 06m 30s]
val_loss: 0.32646316289901733

Best val_loss So Far: 0.2351001873612404
Total elapsed time: 06h 32m 08s
INFO:tensorflow:Oracle triggered exit


In [13]:

tuner.results_summary()


Results summary
Results in tuner2/untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
units: 320
activation: tanh
2nd_layer: True
learning_rate: 0.001
units2: 128
tuner/epochs: 100
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.2351001873612404
Trial summary
Hyperparameters:
units: 384
activation: sigmoid
2nd_layer: True
learning_rate: 0.001
units2: 448
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 4
tuner/round: 4
tuner/trial_id: 0144
Score: 0.23580218106508255
Trial summary
Hyperparameters:
units: 480
activation: sigmoid
2nd_layer: True
learning_rate: 0.001
units2: 64
tuner/epochs: 34
tuner/initial_epoch: 12
tuner/bracket: 4
tuner/round: 3
tuner/trial_id: 0137
Score: 0.23920027166604996
Trial summary
Hyperparameters:
units: 256
activation: tanh
2nd_layer: True
learning_rate: 0.001
units2: 192
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0228
Score: 0.24101244658231735
Trial summary
Hyperparame

In [14]:
tuner.search_space_summary()


Search space summary
Default search space size: 6
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation (Choice)
{'default': 'tanh', 'conditions': [], 'values': ['tanh', 'sigmoid', 'relu'], 'ordered': False}
2nd_layer (Boolean)
{'default': False, 'conditions': []}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units2 (Int)
{'default': None, 'conditions': [{'class_name': 'Parent', 'config': {'name': '2nd_layer', 'values': [1]}}], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation (Choice)
{'default': 'tanh', 'conditions': [{'class_name': 'Parent', 'config': {'name': '2nd_layer', 'values': [1]}}], 'values': ['tanh', 'sigmoid', 'relu'], 'ordered': False}


Best val_loss So Far: 0.3011022210121155

- 5 epochs
- 512 units
- tanh activation
- True 2nd layer
- 0.001 learning_rate
- 192 units2


In [25]:
results

,loss,accuracy
Baseline,0.285855,0.895535


In [32]:
params = tuner.get_best_hyperparameters(10)

for i, param in enumerate(params):
    print('Running model:', i)
    model = tuner.hypermodel.build(param)
    model.fit(x_train, y_train, epochs=param.values.get('tuner/epochs'), callbacks=callbacks, validation_data=(x_val, y_val))
    hyper_df = evaluate_model(model, x_test, y_test)
    hyper_df.index = ["Hypertuned-" + str(i)]
    hyper_df['units'] = param.values.get('units')
    hyper_df['activation'] = param.values.get('activation')
    hyper_df['learning_rate'] = param.values.get('learning_rate')
    hyper_df['epochs'] = param.values.get('tuner/epochs')
    hyper_df['2nd_layer'] = param.values.get('2nd_layer')
    hyper_df['units2'] = param.values.get('units2')
    # Append results in dataframe
    results = pd.concat([results, hyper_df])

Running model: 0
Epoch 1/100


2023-02-24 12:06:15.838395: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


347/347 [==============================] - ETA: 0s - loss: 0.6015 - accuracy: 0.7454

2023-02-24 12:06:23.045103: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Assets written to: ./checkpoints/model_1/assets
347/347 [==============================] - 10s 26ms/step - loss: 0.6015 - accuracy: 0.7454 - val_loss: 0.5650 - val_accuracy: 0.7750
Epoch 2/100
347/347 [==============================] - 8s 24ms/step - loss: 0.4316 - accuracy: 0.8254 - val_loss: 0.3760 - val_accuracy: 0.8424
Epoch 3/100
347/347 [==============================] - 8s 23ms/step - loss: 0.3824 - accuracy: 0.8496 - val_loss: 0.3397 - val_accuracy: 0.8692
Epoch 4/100
347/347 [==============================] - 8s 24ms/step - loss: 0.3461 - accuracy: 0.8619 - val_loss: 0.3343 - val_accuracy: 0.8709
Epoch 5/100
347/347 [==============================] - 8s 23ms/step - loss: 0.3209 - accuracy: 0.8748 - val_loss: 0.3104 - val_accuracy: 0.8774
Epoch 6/100
347/347 [==============================] - 8s 23ms/step - loss: 0.3067 - accuracy: 0.8800 - val_loss: 0.3466 - val_accuracy: 0.8641
Epoch 7/100
347/347 [==============================] - 8s 23ms/step - loss: 0.2916 

...

INFO:tensorflow:Assets written to: ./checkpoints/model_1/assets
347/347 [==============================] - 10s 27ms/step - loss: 0.6437 - accuracy: 0.7254 - val_loss: 0.4965 - val_accuracy: 0.8116
Epoch 2/100
347/347 [==============================] - 8s 24ms/step - loss: 0.4602 - accuracy: 0.8239 - val_loss: 0.4154 - val_accuracy: 0.8517
Epoch 3/100
347/347 [==============================] - 8s 23ms/step - loss: 0.3978 - accuracy: 0.8489 - val_loss: 0.4206 - val_accuracy: 0.8319
Epoch 4/100
347/347 [==============================] - 8s 23ms/step - loss: 0.3499 - accuracy: 0.8724 - val_loss: 0.3611 - val_accuracy: 0.8603
Epoch 5/100
347/347 [==============================] - 8s 23ms/step - loss: 0.3367 - accuracy: 0.8733 - val_loss: 0.3830 - val_accuracy: 0.8579
Epoch 6/100
347/347 [==============================] - 8s 24ms/step - loss: 0.3171 - accuracy: 0.8825 - val_loss: 0.3355 - val_accuracy: 0.8720
Epoch 7/100
347/347 [==============================] - 8s 24ms/step - loss: 0.2978 

2023-02-24 13:38:38.346421: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


347/347 [==============================] - ETA: 0s - loss: 0.6581 - accuracy: 0.7083

2023-02-24 13:38:45.350734: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Assets written to: ./checkpoints/model_1/assets
347/347 [==============================] - 10s 26ms/step - loss: 0.6581 - accuracy: 0.7083 - val_loss: 0.4875 - val_accuracy: 0.8129
Epoch 2/34
347/347 [==============================] - 8s 24ms/step - loss: 0.4393 - accuracy: 0.8221 - val_loss: 0.3942 - val_accuracy: 0.8438
Epoch 3/34
347/347 [==============================] - 8s 23ms/step - loss: 0.3773 - accuracy: 0.8531 - val_loss: 0.3422 - val_accuracy: 0.8663
Epoch 4/34
347/347 [==============================] - 8s 24ms/step - loss: 0.3573 - accuracy: 0.8577 - val_loss: 0.3232 - val_accuracy: 0.8649
Epoch 5/34
347/347 [==============================] - 8s 23ms/step - loss: 0.3258 - accuracy: 0.8707 - val_loss: 0.3141 - val_accuracy: 0.8793
Epoch 6/34
347/347 [==============================] - 8s 24ms/step - loss: 0.3077 - accuracy: 0.8796 - val_loss: 0.4022 - val_accuracy: 0.8619
Epoch 7/34
347/347 [==============================] - 8s 23ms/step - loss: 0.3062 - accu

Model 1: 116/116 [==============================] - 3s 24ms/step - loss: 0.2007 - accuracy: 0.9369


In [33]:
results

,loss,accuracy,units,activation,learning_rate,epochs,2nd_layer,units2
Baseline,0.285855,0.895535,NaN,NaN,NaN,NaN,NaN,NaN
Hypertuned-0,0.207007,0.929635,320.0,tanh,0.001,100.0,True,128.0
Hypertuned-1,0.219640,0.937754,384.0,sigmoid,0.001,100.0,True,448.0
Hypertuned-2,0.222227,0.920433,480.0,sigmoid,0.001,34.0,True,64.0
Hypertuned-3,0.220833,0.929635,256.0,tanh,0.001,100.0,True,192.0
Hypertuned-4,0.234431,0.923681,160.0,sigmoid,0.001,100.0,True,96.0
Hypertuned-5,0.244205,0.909608,288.0,tanh,0.001,34.0,True,192.0
Hypertuned-6,0.208075,0.926928,384.0,sigmoid,0.001,34.0,True,448.0
Hypertuned-7,0.229467,0.919351,320.0,sigmoid,0.001,34.0,True,96.0
Hypertuned-8,0.217183,0.934506,480.0,sigmoid,0.001,100.0,True,64.0


In [12]:
model = build_model(params)


In [13]:
model.fit(x_train, y_train, callbacks=callbacks, epochs=25, validation_data=(x_val, y_val))


Epoch 1/25


2023-02-23 10:20:57.846891: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


345/347 [============================>.] - ETA: 0s - loss: 0.8801 - accuracy: 0.7524

2023-02-23 10:21:01.600395: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Assets written to: ./checkpoints/model_1/assets
347/347 [==============================] - 5s 14ms/step - loss: 0.8789 - accuracy: 0.7527 - val_loss: 0.4642 - val_accuracy: 0.8281
Epoch 2/25
347/347 [==============================] - 5s 15ms/step - loss: 0.4610 - accuracy: 0.8316 - val_loss: 0.4378 - val_accuracy: 0.8579
Epoch 3/25
347/347 [==============================] - 5s 14ms/step - loss: 0.4326 - accuracy: 0.8462 - val_loss: 0.3621 - val_accuracy: 0.8730
Epoch 4/25
347/347 [==============================] - 4s 13ms/step - loss: 0.3908 - accuracy: 0.8599 - val_loss: 0.4168 - val_accuracy: 0.8714
Epoch 5/25
347/347 [==============================] - 4s 13ms/step - loss: 0.3839 - accuracy: 0.8637 - val_loss: 0.3977 - val_accuracy: 0.8373
Epoch 6/25
347/347 [==============================] - 4s 13ms/step - loss: 0.3721 - accuracy: 0.8670 - val_loss: 0.3695 - val_accuracy: 0.8757
Epoch 7/25
347/347 [==============================] - 4s 13ms/step - loss: 0.3643 - accur

In [14]:
# Evaluate the model: You should evaluate the performance of the model
# on the test set and compare it to the training set performance to determine
# if the model has overfitted or underfitted the data.
loss, acc = model.evaluate(x_test, y_test)

print("loss: %.2f" % loss)
print("acc: %.2f" % acc)


116/116 [==============================] - 1s 9ms/step - loss: 0.2923 - accuracy: 0.8980
loss: 0.29
acc: 0.90


In [15]:
r = model.predict(x_test)
# res = []
# tmp = []
# for i in range(24):
#   if i%3 ==0 and i !=0:
#     res.append(tmp)
#     tmp = []
#   tmp.append(r[i])
# res.append(tmp)
# res
res = [np.argmax(i) for i in r]


 85/116 [====================>.........] - ETA: 0s

2023-02-23 10:23:06.104417: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


116/116 [==============================] - 0s 2ms/step


In [16]:
model.save('./out/model')


INFO:tensorflow:Assets written to: ./out/model/assets


In [17]:
# Convert to TFLite model
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(
    './out/model')  # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


2023-02-23 10:23:23.750601: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-02-23 10:23:23.750760: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-02-23 10:23:23.754149: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: ./out/model
2023-02-23 10:23:23.755967: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2023-02-23 10:23:23.755972: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: ./out/model
2023-02-23 10:23:23.758364: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2023-02-23 10:23:23.758820: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-02-23 10:23:23.794249: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: ./out/model
2023-02-23 10:23:23.79